# Filter SNPs

To perform EBV DNAemia GWAS within the AoU cohort of individuals with European ancestry (EUR), we followed the demo workspace for performing GWAS in AoU: https://workbench.researchallofus.org/workspaces/aou-rw-5981f9dc/aouldlgwasregeniedsubctv6duplicate/analysis. In particular, we modified the code from the `4.0_regenie_dsub_HP_TM` script to run REGENIE with our EBV DNA binary trait (from the `EBV_DNA_Quantification` folder). 

REGENIE also requires input SNPlists for step1 and 2. Since REGENIE errors out when encountering a variant with variance around 0, we used the Variant Annotation Table (VAT) (https://support.researchallofus.org/hc/en-us/articles/4615256690836-Variant-Annotation-Table) to filter variants in individuals with EUR ancestry. Specifically, we employed a broad filter of alternative allele frequency (AF) of $0 < AF < 0.49$ or $0.51 < AF < 0.99$ (). We also filtered for variants with sample counts () of at least 100. 

To do this, we created a Controlled Tier workspace (storage space 250 GB) and ran the following commands in the terminal:
- Copy over the VAT dataset (~115.6 GiB):
```bash
gsutil -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/vat/vat_complete_v7.1.bgz.tsv.gz . 
```
- The locus column is column 1. The EUR ancestry-relevant columns are columns 28-31 (gvs_eur_ac, gvs_eur_an, gvs_eur_af, gvs_eur_sc). Confirm this is correct:
```bash
zcat < vat_complete_v7.1.bgz.tsv.gz | head -n 100 | awk -F'\t' '{print $1"\t"$28"\t"$29"\t"$30"\t"$31}'
```
- Filter for variants with at least 100 individuals:
```bash
zcat < vat_complete_v7.1.bgz.tsv.gz | awk -F'\t' '$31 >= 100 {print $1"\t"$28"\t"$29"\t"$30"\t"$31}' > EUR_SNP_stats_100
```
- Filter for variants that will definitely not have variance too close to 0 (gvs_eur_af not too close to 0, 0.5, or 1):
```bash
cat EUR_SNP_stats_100 | awk -F'\t' '$4 > 0.01 && $4 < 0.99 && ($4 > 0.51 || $4 < 0.49) {print $1"\t"$2"\t"$3"\t"$4"\t"$5}' > EUR_SNP_stats_100_filt
```
- Get unique variant loci and zip the file:
```bash
cat EUR_SNP_stats_100_filt | sort | uniq | awk -F'\t' '{print $1}' > EUR_SNP_stats_100_filt_snps.txt

gzip -c EUR_SNP_stats_100_filt_snps.txt > EUR_SNP_stats_100_filt_snps.txt.gz
```

## Process SNPs into REGENIE input format

Move the zipped SNPlist into the GWAS workspace (in the following scripts referred to as `ebvgwas`). Next, split the SNPs by chromosome 1-22. REGENIE's step1 takes in <= 1M variants (unless forced, which is not recommended). Since chr2 has > 1M SNPs, we split the file by the centromere midpoint, position 93139351 (the midpoint of 92188145 and 94090557, the chr2 centromere coordinates for hg38 according to UCSC).

This was done with the following commands in the terminal:
- Split by chromosome:
```bash
mkdir AOU_SNPs_EUR && cd AOU_SNPs_EUR && zcat < ../EUR_SNP_stats_100_filt_snps.txt.gz | awk '{split($1, a, "-"); if (a[1] >= 1 && a[1] <= 23) {gsub("-", ":", $1); print "chr" $1 > ("chr" a[1] "_snplist.txt")}}'
```
- Get number of SNPs per chromosome:
```bash
wc -l chr*
```
- Split chr2 by midpoint:
```bash
mv chr2_snplist.txt chr2_snplist_all.txt
awk -F':' '$2 <= 93139351 { print > "chr2_snplist.txt" }
         $2 >  93139351 { print > "chr2_snplist_2.txt" }' chr2_snplist_all.txt
rm chr2_snplist_all.txt
```
The SNPlist folder now contains the SNPs for REGENIE input.